In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import fastparquet
import pyarrow
import glob
import dask.dataframe as dd
import os

In [2]:
# Load electricity dataset
elec = pd.read_csv("electricity_cleaned.csv")

# Convert timestamp to datetime
elec["timestamp"] = pd.to_datetime(elec["timestamp"])

In [3]:
elec.shape

(17544, 1579)

In [4]:
elec.head()

,timestamp,Panther_parking_Lorriane,Panther_lodging_Cora,Panther_office_Hannah,Panther_lodging_Hattie,Panther_education_Teofila,Panther_education_Jerome,Panther_retail_Felix,Panther_parking_Asia,Panther_education_Misty,...,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Buddy,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Ileana,Mouse_health_Estela,Mouse_science_Micheal
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,123.200,727.575,69.200,8.8224,370.0870,10.0,282.9965,26.0,135.0,168.2243
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,126.475,731.200,66.275,17.6449,737.8260,30.0,574.9265,51.0,265.0,336.4486
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.825,724.675,64.675,17.6449,729.9255,30.0,570.2780,50.0,272.0,336.4486
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.475,737.375,65.875,17.6449,722.2620,20.0,561.1470,52.0,276.0,336.4486
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.675,721.150,66.275,17.6449,719.1665,30.0,564.3695,50.0,280.0,336.4486


In [5]:
# Convert to Parquet file
elec.to_parquet("electricity_cleaned.parquet", engine="fastparquet", index=False)
print("✅ Saved electricity_cleaned.parquet")

✅ Saved electricity_cleaned.parquet


In [6]:
# Load just the first row to get column names
elec_cols = pd.read_parquet("electricity_cleaned.parquet", engine="fastparquet", columns=None).columns

# First column should be timestamp
timestamp_col = "timestamp"
building_cols = [c for c in elec_cols if c != timestamp_col]

print(f"Total buildings: {len(building_cols)}")

Total buildings: 1578


In [12]:
def check_missing_in_batches(parquet_file, building_cols, batch_size=100):
    results = []

    for i in range(0, len(building_cols), batch_size):
        batch = building_cols[i:i+batch_size]

        # Load batch of buildings + timestamp
        df = pd.read_parquet(parquet_file, engine="fastparquet", columns=[timestamp_col] + batch)

        # Calculate null percentages
        null_percent = df.isna().mean() * 100
        null_percent = null_percent.drop(timestamp_col)  # remove timestamp

        # Collect results
        results.append(null_percent)

        print(f"✅ Processed batch {i//batch_size + 1}, buildings {i}–{i+len(batch)-1}")

    # Combine results into one Series
    all_nulls = pd.concat(results)
    return all_nulls


In [13]:
missing_perc = check_missing_in_batches("electricity_cleaned.parquet", building_cols, batch_size=100)

✅ Processed batch 1, buildings 0–99
✅ Processed batch 2, buildings 100–199
✅ Processed batch 3, buildings 200–299
✅ Processed batch 4, buildings 300–399
✅ Processed batch 5, buildings 400–499
✅ Processed batch 6, buildings 500–599
✅ Processed batch 7, buildings 600–699
✅ Processed batch 8, buildings 700–799
✅ Processed batch 9, buildings 800–899
✅ Processed batch 10, buildings 900–999
✅ Processed batch 11, buildings 1000–1099
✅ Processed batch 12, buildings 1100–1199
✅ Processed batch 13, buildings 1200–1299
✅ Processed batch 14, buildings 1300–1399
✅ Processed batch 15, buildings 1400–1499
✅ Processed batch 16, buildings 1500–1577


In [14]:
missing_perc.head()

Panther_parking_Lorriane     19.277246
Panther_lodging_Cora         19.100547
Panther_office_Hannah        19.322845
Panther_lodging_Hattie       19.311446
Panther_education_Teofila    19.311446
dtype: float64

In [15]:
missing_perc.sample(10)

Hog_assembly_Letha             2.103283
Eagle_education_Luther         0.473096
Bobcat_education_Barbra       99.373005
Gator_other_Reginald           8.492932
Mouse_science_Micheal         58.663931
Cockatoo_lodging_Aimee         8.150935
Cockatoo_assembly_Genoveva     8.185135
Cockatoo_office_Alton          8.122435
Panther_lodging_Cora          19.100547
Hog_education_Hallie           0.011400
dtype: float64

In [16]:
missing_perc.shape

(1578,)

In [20]:
# Filter out buildings with more than 10% missing values
threshold = 10
valid_buildings = missing_perc[missing_perc <= threshold].index.tolist()

print(f" Kept {len(valid_buildings)} / {len(building_cols)} buildings (≤ {threshold}% missing)")

 Kept 1256 / 1578 buildings (≤ 10% missing)


In [21]:
# Reload only valid buildings + timestamp
elec_filtered = pd.read_parquet("electricity_cleaned.parquet", 
                                engine="fastparquet", 
                                columns=[timestamp_col] + valid_buildings)

elec_filtered.to_parquet("electricity_filtered.parquet", engine="fastparquet", index=False)
print("✅ Saved electricity_filtered.parquet")

✅ Saved electricity_filtered.parquet


In [7]:
elec_filt = elec_cols = pd.read_parquet("electricity_filtered.parquet", engine="fastparquet")

In [9]:
elec_filt.head()

,timestamp,Robin_public_Carolina,Robin_lodging_Dorthy,Robin_education_Zenia,Robin_office_Dina,Robin_education_Derick,Robin_education_Della,Robin_education_Leslie,Robin_office_Sammie,Robin_education_Mercedes,...,Cockatoo_public_Leah,Cockatoo_education_Amira,Cockatoo_public_Harland,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Estela
0,2016-01-01 00:00:00,36.438,14.3,52.25,13.3776,229.0,0.3746,324.75,57.75,80.930,...,93.100,38.025,26.200,123.200,727.575,69.200,370.0870,10.0,282.9965,135.0
1,2016-01-01 01:00:00,70.750,27.3,106.25,26.7551,449.5,0.7492,624.75,72.00,166.359,...,95.825,38.175,25.250,126.475,731.200,66.275,737.8260,30.0,574.9265,265.0
2,2016-01-01 02:00:00,74.312,25.7,104.25,26.7551,451.5,0.7492,626.50,66.25,164.868,...,91.925,38.125,23.850,127.825,724.675,64.675,729.9255,30.0,570.2780,272.0
3,2016-01-01 03:00:00,73.438,22.4,107.75,26.7551,445.0,0.7492,630.25,70.00,160.867,...,91.775,37.800,24.400,130.475,737.375,65.875,722.2620,20.0,561.1470,276.0
4,2016-01-01 04:00:00,70.313,20.1,104.50,26.7551,464.0,0.7492,635.00,68.75,158.851,...,90.450,37.975,28.275,129.675,721.150,66.275,719.1665,30.0,564.3695,280.0


In [10]:
elec_filt.columns

Index(['timestamp', 'Robin_public_Carolina', 'Robin_lodging_Dorthy',
       'Robin_education_Zenia', 'Robin_office_Dina', 'Robin_education_Derick',
       'Robin_education_Della', 'Robin_education_Leslie',
       'Robin_office_Sammie', 'Robin_education_Mercedes',
       ...
       'Cockatoo_public_Leah', 'Cockatoo_education_Amira',
       'Cockatoo_public_Harland', 'Cockatoo_public_Caleb',
       'Cockatoo_education_Tyler', 'Cockatoo_public_Shad',
       'Mouse_health_Modesto', 'Mouse_lodging_Vicente', 'Mouse_health_Justin',
       'Mouse_health_Estela'],
      dtype='object', length=1257)

In [11]:
elec_filt = elec_filt.set_index("timestamp")
elec_interp = elec_filt.interpolate(method="time")

In [12]:
elec_interp = elec_interp.fillna(method="ffill").fillna(method="bfill")
elec_interp = elec_interp.reset_index()

/tmp/ipykernel_9976/386540496.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  elec_interp = elec_interp.fillna(method="ffill").fillna(method="bfill")


In [13]:
elec_interp.sample(15)

,timestamp,Robin_public_Carolina,Robin_lodging_Dorthy,Robin_education_Zenia,Robin_office_Dina,Robin_education_Derick,Robin_education_Della,Robin_education_Leslie,Robin_office_Sammie,Robin_education_Mercedes,...,Cockatoo_public_Leah,Cockatoo_education_Amira,Cockatoo_public_Harland,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Estela
8957,2017-01-08 05:00:00,51.2500,19.7,187.9000,32.0000,430.5000,3.0000,707.6250,58.5000,156.6570,...,104.266700,31.000000,39.30000,119.33330,707.800000,107.100000,244.151,30.0,662.791,303.0
10820,2017-03-26 20:00:00,131.9850,51.3,214.4000,30.0000,483.5000,3.0000,800.0000,95.5000,181.6870,...,199.875000,37.775000,46.05000,225.60000,758.225000,150.425000,303.282,40.0,634.329,339.0
9905,2017-02-16 17:00:00,170.7500,40.8,274.0000,53.0000,604.0000,10.0000,1121.7500,121.0000,265.5940,...,195.925000,102.225000,59.85000,251.12500,826.050000,164.675000,548.504,60.0,543.159,537.0
17128,2017-12-14 16:00:00,167.2854,33.3,261.6000,33.0958,631.8000,7.2324,853.2261,126.7000,240.5849,...,194.750000,40.750000,62.77500,201.45000,790.500000,163.875000,297.167,30.0,685.731,308.0
13414,2017-07-12 22:00:00,150.7500,34.2,229.0000,24.5000,451.4170,3.0000,827.7500,66.4000,260.9140,...,140.900000,37.950000,41.17500,109.50000,700.900000,84.825000,422.338,50.0,797.570,323.0
4956,2016-07-25 12:00:00,142.5000,33.7,222.5000,44.0000,784.0000,0.7492,1176.5000,86.2500,348.5150,...,215.375000,71.500000,62.25000,253.22500,864.450000,205.800000,641.875,60.0,929.182,540.0
5373,2016-08-11 21:00:00,118.0620,35.3,162.2500,26.5000,631.0000,0.7492,811.0000,67.7500,290.9370,...,102.450000,39.775000,38.47500,123.87500,783.850000,110.825000,355.291,40.0,1027.941,378.0
2013,2016-03-24 21:00:00,156.6694,38.5,142.2721,34.2212,516.5000,0.7492,833.5311,87.8658,231.0183,...,168.461588,34.980429,49.98618,223.29515,785.489957,172.790558,764.005,40.0,633.673,339.0
6081,2016-09-10 09:00:00,110.8120,30.9,155.2500,26.0000,637.0000,0.7492,844.4530,62.7500,245.6400,...,154.100000,42.850000,44.77500,138.90000,770.900000,97.325000,587.195,60.0,919.476,387.0
7741,2016-11-18 13:00:00,188.6870,41.5,279.5000,56.5000,695.0000,9.0000,1192.8750,115.1000,321.6640,...,208.325000,63.625000,75.00000,274.17500,851.350000,160.200000,405.854,50.0,947.335,545.0


In [19]:
# Add a date column ignoring hours
elec_interp["date"] = pd.to_datetime(elec_interp["timestamp"].dt.date)

In [20]:
# Group by date and sum across buildings and drop timestamp
daily_elec = elec_interp.drop(columns=["timestamp"]).groupby("date").sum().reset_index()

print("✅ Daily dataset shape:", daily_elec.shape)

✅ Daily dataset shape: (731, 1257)


In [21]:
daily_elec.head()

,date,Robin_public_Carolina,Robin_lodging_Dorthy,Robin_education_Zenia,Robin_office_Dina,Robin_education_Derick,Robin_education_Della,Robin_education_Leslie,Robin_office_Sammie,Robin_education_Mercedes,...,Cockatoo_public_Leah,Cockatoo_education_Amira,Cockatoo_public_Harland,Cockatoo_public_Caleb,Cockatoo_education_Tyler,Cockatoo_public_Shad,Mouse_health_Modesto,Mouse_lodging_Vicente,Mouse_health_Justin,Mouse_health_Estela
0,2016-01-01,2183.750,805.4,2852.25,628.7449,11219.5,17.6062,16130.25,1739.00,3911.922,...,2208.900,896.350,602.650,3108.400,17824.3500,1851.925,18601.5150,790.0,14740.9715,6971.0
1,2016-01-02,2205.563,789.3,2761.75,649.7552,11483.5,17.9808,16402.00,1594.25,4159.094,...,2358.350,895.925,582.025,3130.375,17922.2501,1970.825,19081.0505,810.0,16045.5440,7257.0
2,2016-01-03,2108.750,795.7,2712.25,618.0000,11440.5,17.9808,16314.75,1791.25,4048.469,...,3428.375,896.125,786.875,3183.800,17874.6498,2177.550,19042.6115,820.0,15446.4870,7435.0
3,2016-01-04,3536.562,778.2,3253.75,809.0000,13052.0,17.9808,17657.50,2189.75,4671.281,...,4719.475,932.750,1266.975,4798.500,18875.4000,4066.100,21577.2485,930.0,17019.1110,9074.0
4,2016-01-05,3771.376,755.8,3375.00,771.5000,13434.5,17.9808,18657.25,2356.00,4243.156,...,4859.750,931.275,1309.675,4887.725,19123.8750,4158.525,22031.0430,1030.0,16801.8090,9514.0


In [22]:
daily_elec.to_parquet("electricity_daily.parquet", engine="fastparquet", index=False)

In [23]:
# Load daily data
daily_elec = pd.read_parquet("electricity_daily.parquet", engine="fastparquet")

# Melt wide to long
elec_long = daily_elec.melt(id_vars=["date"], 
                            var_name="building_id", 
                            value_name="daily_consumption")

print("✅ Long format shape:", elec_long.shape)

✅ Long format shape: (918136, 3)


In [24]:
elec_long.head()

,date,building_id,daily_consumption
0,2016-01-01,Robin_public_Carolina,2183.750
1,2016-01-02,Robin_public_Carolina,2205.563
2,2016-01-03,Robin_public_Carolina,2108.750
3,2016-01-04,Robin_public_Carolina,3536.562
4,2016-01-05,Robin_public_Carolina,3771.376


In [25]:
elec_long.to_parquet("electricity_daily_long.parquet", engine="fastparquet", index=False)
print("✅ Saved electricity_daily_long.parquet")

✅ Saved electricity_daily_long.parquet


In [26]:
# Load metadata
meta = pd.read_csv("metadata.csv")

print(" Shape:", meta.shape)

 Shape: (1636, 32)


In [27]:
meta.head()

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
0,Panther_lodging_Dean,Panther,NaN,0.0,Lodging/residential,Residence Hall,508.8,5477.0,28.517689,-81.379039,...,1989.0,NaN,NaN,NaN,NaN,271,NaN,NaN,NaN,NaN
1,Panther_lodging_Shelia,Panther,NaN,0.0,Lodging/residential,Residence Hall,929.0,10000.0,28.517689,-81.379039,...,1992.0,NaN,NaN,NaN,NaN,62,NaN,NaN,NaN,NaN
2,Panther_lodging_Ricky,Panther,NaN,0.0,Lodging/residential,Residence Hall,483.1,5200.0,28.517689,-81.379039,...,2016.0,NaN,NaN,NaN,NaN,534,NaN,NaN,NaN,NaN
3,Panther_education_Rosalie,Panther,0.0,0.0,Education,Research,690.5,7432.0,28.517689,-81.379039,...,2008.0,NaN,NaN,NaN,NaN,276,NaN,NaN,NaN,NaN
4,Panther_education_Misty,Panther,1.0,0.0,Education,Research,252.7,2720.0,28.517689,-81.379039,...,2004.0,NaN,NaN,NaN,NaN,375,NaN,NaN,NaN,NaN


In [29]:
meta.sample(15)

,building_id,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,lat,lng,...,yearbuilt,date_opened,numberoffloors,occupants,energystarscore,eui,site_eui,source_eui,leed_level,rating
748,Lamb_lodging_Burt,Lamb,671.0,5.0,Lodging/residential,Dormitory,705.0,7589.0,51.497838,-3.186246,...,1976.0,NaN,3.0,34.0,NaN,NaN,NaN,NaN,NaN,D
209,Fox_education_Kim,Fox,176.0,2.0,Education,College Classroom,5782.1,62238.0,33.424425,-111.928140,...,1970.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1572,Cockatoo_assembly_Fernanda,Cockatoo,1392.0,15.0,Entertainment/public assembly,Fitness Center,7074.8,76153.0,42.459837,-76.485292,...,1963.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,Rat_other_Daphne,Rat,508.0,3.0,Other,Other,10431.4,112283.0,38.903504,-77.005349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,Panther_retail_Kristina,Panther,43.0,0.0,Retail,Retail,5487.9,59071.0,28.517689,-81.379039,...,1980.0,NaN,NaN,NaN,NaN,125,NaN,NaN,NaN,NaN
871,Peacock_office_Glenn,Peacock,NaN,6.0,Office,Office,2179.3,23458.0,40.350000,-74.700000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,Rat_public_Fern,Rat,378.0,3.0,Public services,Fire Station,3274.0,35241.0,38.903504,-77.005349,...,1978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1511,Cockatoo_education_Orlando,Cockatoo,1331.0,15.0,Education,College Laboratory,15887.2,171008.0,42.459837,-76.485292,...,1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,Rat_education_Jena,Rat,445.0,3.0,Education,K-12 School,3804.4,40950.0,38.903504,-77.005349,...,1910.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534,Cockatoo_education_Christi,Cockatoo,1354.0,15.0,Education,College Classroom,2158.3,23232.0,42.459837,-76.485292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Missing value percentages
missing_meta = meta.isna().mean() * 100
missing_meta = missing_meta.sort_values(ascending=False)

print("✅ Missing values (%):")
print(missing_meta)

✅ Missing values (%):
solar                    99.694377
leed_level               99.022005
date_opened              98.716381
irrigation               97.738386
water                    91.075795
source_eui               90.036675
site_eui                 90.036675
energystarscore          90.036675
gas                      89.180929
rating                   88.753056
hotwater                 88.691932
heatingtype              86.858191
occupants                85.941320
eui                      81.723716
steam                    77.383863
numberoffloors           73.044010
chilledwater             66.075795
industry                 64.608802
subindustry              64.608802
yearbuilt                50.061125
lng                      14.486553
lat                      14.486553
building_id_kaggle       11.430318
electricity               3.545232
site_id_kaggle            2.261614
primaryspaceusage         1.283619
sub_primaryspaceusage     1.283619
timezone                  0.00000

In [30]:
# Set threshold (e.g. drop if >50% missing)
threshold = 15
cols_to_keep = missing_meta[missing_meta <= threshold].index.tolist()

meta_filtered = meta[cols_to_keep]

print(f" Kept {len(cols_to_keep)} / {meta.shape[1]} columns (≤ {threshold}% missing)")
print("New shape:", meta_filtered.shape)

 Kept 12 / 32 columns (≤ 15% missing)
New shape: (1636, 12)


In [31]:
# Columns to drop
drop_cols = ["building_id_kaggle", "site_id_kaggle", "electricity"]

In [39]:
# Drop if they exist
meta_filtered = meta_filtered.drop(columns=[c for c in drop_cols if c in meta_filtered.columns])

In [40]:
meta_filtered.head()

,lng,lat,primaryspaceusage,sub_primaryspaceusage,timezone,sqm,sqft,site_id,building_id
0,-81.379039,28.517689,Lodging/residential,Residence Hall,US/Eastern,508.8,5477.0,Panther,Panther_lodging_Dean
1,-81.379039,28.517689,Lodging/residential,Residence Hall,US/Eastern,929.0,10000.0,Panther,Panther_lodging_Shelia
2,-81.379039,28.517689,Lodging/residential,Residence Hall,US/Eastern,483.1,5200.0,Panther,Panther_lodging_Ricky
3,-81.379039,28.517689,Education,Research,US/Eastern,690.5,7432.0,Panther,Panther_education_Rosalie
4,-81.379039,28.517689,Education,Research,US/Eastern,252.7,2720.0,Panther,Panther_education_Misty


In [41]:
meta_filtered = meta_filtered[meta_filtered.columns[::-1]]

In [42]:
meta_filtered.head()

,building_id,site_id,sqft,sqm,timezone,sub_primaryspaceusage,primaryspaceusage,lat,lng
0,Panther_lodging_Dean,Panther,5477.0,508.8,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
1,Panther_lodging_Shelia,Panther,10000.0,929.0,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
2,Panther_lodging_Ricky,Panther,5200.0,483.1,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
3,Panther_education_Rosalie,Panther,7432.0,690.5,US/Eastern,Research,Education,28.517689,-81.379039
4,Panther_education_Misty,Panther,2720.0,252.7,US/Eastern,Research,Education,28.517689,-81.379039


In [43]:
meta_filtered.to_csv("metadata_filtered.csv", index=False)
print("Saved metadata_filtered.csv (for manual editing)")

Saved metadata_filtered.csv (for manual editing)


In [44]:
# Edited File loaded back in
meta_filtered = pd.read_csv("metadata_filtered.csv")

In [45]:
meta_filtered.head()

,building_id,site_id,sqft,sqm,timezone,sub_primaryspaceusage,primaryspaceusage,lat,lng
0,Panther_lodging_Dean,Panther,5477,508.8,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
1,Panther_lodging_Shelia,Panther,10000,929.0,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
2,Panther_lodging_Ricky,Panther,5200,483.1,US/Eastern,Residence Hall,Lodging/residential,28.517689,-81.379039
3,Panther_education_Rosalie,Panther,7432,690.5,US/Eastern,Research,Education,28.517689,-81.379039
4,Panther_education_Misty,Panther,2720,252.7,US/Eastern,Research,Education,28.517689,-81.379039


In [46]:
# for Building sites with an anonymous location, has location is added to confirm if lat/long is given.
meta_filtered["has_location"] = meta_filtered["lat"].notna().astype(int)

In [47]:
meta_filtered.sample(10)

,building_id,site_id,sqft,sqm,timezone,sub_primaryspaceusage,primaryspaceusage,lat,lng,has_location
490,Rat_education_Dana,Rat,70800,6577.5,US/Eastern,K-12 School,Education,38.903504,-77.005349,1
710,Bear_education_Wilton,Bear,304333,28273.4,US/Pacific,Education,Education,37.871903,-122.260729,1
182,Robin_office_Serena,Robin,30495,2833.1,Europe/London,Office,Office,51.518790,-0.134556,1
1016,Bull_lodging_Terence,Bull,40697,3780.9,US/Central,Dormitory,Lodging/residential,30.267200,-97.743100,1
307,Fox_office_Zachary,Fox,140345,13038.5,US/Mountain,Office,Office,33.424425,-111.928140,1
1112,Bull_education_Joseph,Bull,167055,15519.9,US/Central,College Classroom,Education,30.267200,-97.743100,1
143,Robin_education_Karyl,Robin,32206,2992.0,Europe/London,College Laboratory,Education,51.518790,-0.134556,1
1167,Crow_education_Kate,Crow,68030,6320.2,US/Eastern,Academic,Education,45.387600,-75.696000,1
876,Peacock_office_Jonathon,Peacock,37485,3482.5,US/Eastern,Office,Office,40.350000,-74.700000,1
1439,Eagle_education_Norah,Eagle,97838,9089.4,US/Eastern,Research,Education,NaN,NaN,0


In [48]:
# Save to parquet
meta_filtered.to_parquet("metadata_clean.parquet", engine="fastparquet", index=False)
print("Saved metadata_clean.parquet")

Saved metadata_clean.parquet


Begin Mergin Process of Electricity and Building Metadata

In [49]:
# 1. Load electricity daily (already in long format)
elec_long = pd.read_parquet("electricity_daily_long.parquet", engine="fastparquet")

# 2. Load metadata (already cleaned manually and saved as CSV)
meta_clean = pd.read_parquet("metadata_clean.parquet", engine="fastparquet")

In [50]:
# 3. Merge on building_id
merged = elec_long.merge(meta_clean, on="building_id", how="left")

In [51]:
merged.shape

(918136, 12)

In [52]:
merged.head()

,date,building_id,daily_consumption,site_id,sqft,sqm,timezone,sub_primaryspaceusage,primaryspaceusage,lat,lng,has_location
0,2016-01-01,Robin_public_Carolina,2183.750,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1
1,2016-01-02,Robin_public_Carolina,2205.563,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1
2,2016-01-03,Robin_public_Carolina,2108.750,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1
3,2016-01-04,Robin_public_Carolina,3536.562,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1
4,2016-01-05,Robin_public_Carolina,3771.376,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1


In [53]:
# 4. Save to Parquet
merged.to_parquet("elec_meta_merged.parquet", engine="fastparquet", index=False)
print("Saved elec_meta_merged.parquet")

Saved elec_meta_merged.parquet


In [2]:
# 1. Load weather data
weather = pd.read_csv("weather.csv")
print("Shape:", weather.shape)
print("Columns:", weather.columns.tolist())

Shape: (331166, 10)
Columns: ['timestamp', 'site_id', 'airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']


In [3]:
weather.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
0,2016-01-01 00:00:00,Panther,19.4,NaN,19.4,0.0,NaN,NaN,0.0,0.0
1,2016-01-01 01:00:00,Panther,21.1,6.0,21.1,-1.0,NaN,1019.4,0.0,0.0
2,2016-01-01 02:00:00,Panther,21.1,NaN,21.1,0.0,NaN,1018.8,210.0,1.5
3,2016-01-01 03:00:00,Panther,20.6,NaN,20.0,0.0,NaN,1018.1,0.0,0.0
4,2016-01-01 04:00:00,Panther,21.1,NaN,20.6,0.0,NaN,1019.0,290.0,1.5


In [4]:
# 3. Check missing values percentage
missing_weather = weather.isna().mean().sort_values(ascending=False) * 100
print("\nMissing values (%):\n", missing_weather)


Missing values (%):
 precipDepth6HR    94.515741
cloudCoverage     51.631810
precipDepth1HR    40.217293
seaLvlPressure     6.529656
windDirection      3.927034
windSpeed          0.173327
dewTemperature     0.099044
airTemperature     0.038651
timestamp          0.000000
site_id            0.000000
dtype: float64


In [5]:
# Drop high-missing columns (>30%)
drop_cols = ["precipDepth1HR", "cloudCoverage", "precipDepth6HR"]
weather = weather.drop(columns=drop_cols)

In [6]:
# convert timestap to datetime
weather["timestamp"] = pd.to_datetime(weather["timestamp"])

In [7]:
weather.head()

,timestamp,site_id,airTemperature,dewTemperature,seaLvlPressure,windDirection,windSpeed
0,2016-01-01 00:00:00,Panther,19.4,19.4,NaN,0.0,0.0
1,2016-01-01 01:00:00,Panther,21.1,21.1,1019.4,0.0,0.0
2,2016-01-01 02:00:00,Panther,21.1,21.1,1018.8,210.0,1.5
3,2016-01-01 03:00:00,Panther,20.6,20.0,1018.1,0.0,0.0
4,2016-01-01 04:00:00,Panther,21.1,20.6,1019.0,290.0,1.5


In [ ]:
# Handle missing values with interpolation in remaining columns 
weather = weather.groupby("site_id").apply(
    lambda group: group.set_index("timestamp").interpolate(method="time").reset_index()
).reset_index(drop=True)

In [10]:
weather.head()

,timestamp,site_id,airTemperature,dewTemperature,seaLvlPressure,windDirection,windSpeed
0,2016-01-01 00:00:00,Bear,4.4,-2.2,1020.9,0.0,0.0
1,2016-01-01 01:00:00,Bear,4.4,-4.4,1020.5,20.0,2.1
2,2016-01-01 02:00:00,Bear,4.4,-6.7,1020.8,20.0,2.1
3,2016-01-01 03:00:00,Bear,4.4,-7.8,1020.7,30.0,2.6
4,2016-01-01 04:00:00,Bear,5.0,-9.4,1020.6,0.0,0.0


In [11]:
# Create a proper date column
weather["date"] = pd.to_datetime(weather["timestamp"]).dt.date

In [12]:
# Drop timestamp (not needed anymore)
weather = weather.drop(columns=["timestamp"])

In [13]:
weather["seaLvlPressure"] = weather["seaLvlPressure"].interpolate()

In [14]:
missing_weather = weather.isna().mean().sort_values(ascending=False) * 100
print("\nMissing values (%):\n", missing_weather)


Missing values (%):
 site_id           0.0
airTemperature    0.0
dewTemperature    0.0
seaLvlPressure    0.0
windDirection     0.0
windSpeed         0.0
date              0.0
dtype: float64


In [15]:
# Aggregate by site_id + date (mean of all numeric columns)
weather_daily = weather.groupby(["site_id", "date"]).mean().reset_index()

In [16]:
print("Weather daily shape:", weather_daily.shape)

Weather daily shape: (13869, 7)


In [17]:
weather_daily.head()

,site_id,date,airTemperature,dewTemperature,seaLvlPressure,windDirection,windSpeed
0,Bear,2016-01-01,6.175000,-5.229167,1020.891667,68.750000,3.070833
1,Bear,2016-01-02,8.087500,-1.404167,1017.687500,76.666667,3.300000
2,Bear,2016-01-03,10.112500,1.708333,1011.491667,91.666667,3.120833
3,Bear,2016-01-04,11.270833,8.283333,1007.020833,106.666667,3.233333
4,Bear,2016-01-05,10.962500,9.208333,1006.000000,174.166667,4.866667


In [18]:
weather_daily.sample(15)

,site_id,date,airTemperature,dewTemperature,seaLvlPressure,windDirection,windSpeed
3434,Crow,2017-05-28,19.337500,11.208333,1009.933333,176.250000,2.920833
6237,Hog,2017-02-03,-9.545833,-17.237500,1031.604167,257.083333,3.545833
5607,Gator,2017-05-15,24.604167,19.333333,1014.085417,86.666667,3.266667
3616,Crow,2017-11-26,-2.300000,-5.431250,1011.143750,245.000000,3.793750
6213,Hog,2017-01-10,-6.991667,-10.291667,1001.983333,230.416667,5.616667
4641,Fox,2016-09-22,26.529167,19.766667,1007.085417,170.000000,3.029167
10676,Rat,2017-04-07,8.516667,-1.658333,1002.966667,288.750000,7.708333
11767,Shrew,2016-04-01,7.850000,2.820833,1019.075000,177.500000,4.112500
5773,Gator,2017-10-28,21.041667,17.912500,1011.566667,93.333333,2.158333
11268,Robin,2016-11-19,4.708333,2.587500,1000.254167,198.333333,3.533333


In [20]:
# Ensure 'date' is proper datetime64
weather_daily["date"] = pd.to_datetime(weather_daily["date"])

In [21]:
print(weather_daily.dtypes)

site_id                   object
date              datetime64[ns]
airTemperature           float64
dewTemperature           float64
seaLvlPressure           float64
windDirection            float64
windSpeed                float64
dtype: object


In [22]:
# Save to parquet
weather_daily.to_parquet("weather_daily.parquet", engine="fastparquet", index=False)
print("Saved weather_daily.parquet")

Saved weather_daily.parquet


In [23]:
# Load datasets
elec_meta = pd.read_parquet("elec_meta_merged.parquet", engine="fastparquet")
weather_daily = pd.read_parquet("weather_daily.parquet", engine="fastparquet")

In [24]:
print("Electricity+Meta shape:", elec_meta.shape)
print("Weather shape:", weather_daily.shape)

Electricity+Meta shape: (918136, 12)
Weather shape: (13869, 7)


In [25]:
# Merge on site_id + date
final_merged = pd.merge(
    elec_meta,
    weather_daily,
    on=["site_id", "date"],
    how="left"   # keep all electricity+meta records, fill weather if missing
)


In [26]:
print(" Final merged dataset shape:", final_merged.shape)

 Final merged dataset shape: (918136, 17)


In [27]:
final_merged.head() 

,date,building_id,daily_consumption,site_id,sqft,sqm,timezone,sub_primaryspaceusage,primaryspaceusage,lat,lng,has_location,airTemperature,dewTemperature,seaLvlPressure,windDirection,windSpeed
0,2016-01-01,Robin_public_Carolina,2183.750,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1,5.387500,3.879167,1016.941667,116.666667,4.470833
1,2016-01-02,Robin_public_Carolina,2205.563,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1,9.783333,9.183333,998.087500,166.666667,7.241667
2,2016-01-03,Robin_public_Carolina,2108.750,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1,7.954167,6.687500,991.837500,187.916667,5.633333
3,2016-01-04,Robin_public_Carolina,3536.562,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1,7.841667,6.508333,982.766667,177.083333,4.145833
4,2016-01-05,Robin_public_Carolina,3771.376,Robin,118231,10984.0,Europe/London,Library,Public services,51.51879,-0.134556,1,7.904167,7.058333,984.712500,175.416667,2.750000


In [28]:
# Save final dataset
final_merged.to_parquet("final_dataset.parquet", engine="fastparquet", index=False)
print(" Saved final_dataset.parquet")

 Saved final_dataset.parquet
